In [55]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.genmod import families
from loguru import logger

In [56]:
df = pd.read_excel("smoke.xlsx")
df.head()

,educ,cigpric,white,age,income,cigs,restaurn,lincome,agesq,lcigpric
0,16.0,60.506,1,46,20000,0,0,9.903487,2116,4.102743
1,16.0,57.883,1,40,30000,0,0,10.308950,1600,4.058424
2,12.0,57.664,1,58,30000,3,0,10.308950,3364,4.054633
3,13.5,57.883,1,30,20000,0,0,9.903487,900,4.058424
4,10.0,58.320,1,17,20000,0,0,9.903487,289,4.065945


#### Transformations

In [57]:
df['log_cigpric'] = np.log(df['cigpric'])
df['log_income'] = np.log(df['income'])
df['age2'] = df['age']**2

In [58]:
poisson_model = smf.glm(
    formula='cigs ~ log_cigpric + log_income + white + educ + age + age2',
    data=df,
    family=families.Poisson()
).fit()

logger.info(f"\n{poisson_model.summary()}")

2025-10-08 21:44:00.320 | INFO     | __main__:<module>:7 - 
                 Generalized Linear Model Regression Results                  
Dep. Variable:                   cigs   No. Observations:                  807
Model:                            GLM   Df Residuals:                      800
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8184.0
Date:                Wed, 08 Oct 2025   Deviance:                       14897.
Time:                        21:44:00   Pearson chi2:                 1.65e+04
No. Iterations:                     5   Pseudo R-squ. (CS):             0.6816
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

### Elasticities of `cig` & `income`

In [59]:
price_elasticity = poisson_model.params['log_cigpric']
logger.info(f"\nPrice elasticity: {price_elasticity}")

2025-10-08 21:44:00.328 | INFO     | __main__:<module>:2 - 
Price elasticity: -0.35525307831881137


In [60]:
income_elasticity = poisson_model.params['log_income']
logger.info(f"\nIncome elasticity: {income_elasticity}")

2025-10-08 21:44:00.338 | INFO     | __main__:<module>:2 - 
Income elasticity: 0.084632685261314


### Fitting Values, Max & Min

In [61]:
df['y_hat'] = poisson_model.fittedvalues

logger.info(f"\nMin fitted value: {df['y_hat'].min()}" 
            f"\nMax fitted value: {df['y_hat'].max()}")

2025-10-08 21:44:00.350 | INFO     | __main__:<module>:3 - 
Min fitted value: 0.5150122521426221
Max fitted value: 18.83566970474354


In [62]:
print(df['cigs'].describe())

count    807.000000
mean       8.686493
std       13.721516
min        0.000000
25%        0.000000
50%        0.000000
75%       20.000000
max       80.000000
Name: cigs, dtype: float64


In [63]:
median_income = df['income'].median()
median_educ = df['educ'].median()
median_age = df['age'].median()
median_age2 = median_age ** 2  
logger.info(f"\nMedian income: {median_income}")

2025-10-08 21:44:00.373 | INFO     | __main__:<module>:5 - 
Median income: 20000.0


In [64]:
p75_cigpric = df['cigpric'].quantile(0.75)
logger.info(f"\n75th percentile cig price: {p75_cigpric}")

2025-10-08 21:44:00.384 | INFO     | __main__:<module>:2 - 
75th percentile cig price: 63.179


In [65]:
baseline = pd.DataFrame({
    'log_cigpric': [np.log(df['cigpric'].median())],  
    'log_income': [np.log(median_income)],
    'white': [1],
    'educ': [median_educ],
    'age': [median_age],
    'age2': [median_age2]
})

In [66]:
baseline_pred = poisson_model.predict(baseline)
logger.info(f"\nPredicted mean cigarettes smoked (baseline): {baseline_pred.iloc[0]}")

2025-10-08 21:44:00.411 | INFO     | __main__:<module>:2 - 
Predicted mean cigarettes smoked (baseline): 12.229488591188288


### Predict with price at **75th** percentile

In [67]:
higher_price = baseline.copy()
higher_price['log_cigpric'] = np.log(p75_cigpric)

higher_price_pred = poisson_model.predict(higher_price)
logger.info(f"\nPredicted mean cigarettes smoked (75th percentile price): {higher_price_pred.iloc[0]}")

2025-10-08 21:44:00.430 | INFO     | __main__:<module>:5 - 
Predicted mean cigarettes smoked (75th percentile price): 12.081676332900786


###  Probability that a person per day smokes 5 or less cigarettes.

In [68]:
import math

lambda_baseline = baseline_pred.iloc[0]
logger.info(f"λ (baseline predicted mean): {lambda_baseline}")

2025-10-08 21:44:00.439 | INFO     | __main__:<module>:4 - λ (baseline predicted mean): 12.229488591188288


In [69]:
prob_5_or_less = sum(
    math.exp(-lambda_baseline) * (lambda_baseline ** k) / math.factorial(k)
    for k in range(0, 6)
)

logger.info(f"P(Y ≤ 5 cigarettes): {prob_5_or_less}")

2025-10-08 21:44:00.449 | INFO     | __main__:<module>:6 - P(Y ≤ 5 cigarettes): 0.017605252622754943
